# Costa Rican Household Poverty Level Prediction
---
## Abstract
This project participated a _kaggle_ competition to predict the poverty level from a limited high dimensional dataset. As the purpose of gaining a high rank from the competition, the project is divided into three parts: exploratory data analysis(EDA), clustering analysis and machine learning. 

In the EDA section, project extract the aggregation of features among the individuals within same household to help the prediction. It also drop the features with high correlations to avoid misleading the models. Some important features(The importance is gained from boosting algorithm) are performed visualising operation.

In clustering analysis, the similarity between cluster is demonstrated. PCA is involved for both visualisation and preprocessing.

In machine learning section, a variety of algorithms are explored. The pipeline of machine learning process is :preprocessing &rarr; model &rarr; predict. The process of finding optimised hyperparameters are built from scratch only for `KNN`, the other algorithms use `sklearn` model standard [cross validate grid parameter search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) procedure. Boosting algorithm is the main force in this competition. The highest ranking came from [LightGBM](https://github.com/Microsoft/LightGBM) algorithm.

In [ ]:
prefix =  "../input/costa-rican-household-poverty-prediction/"

In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
# link pandas and plotly
import cufflinks as cf

init_notebook_mode(connected=True)
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')
%matplotlib inline

from matplotlib import rc

### Predefine Function
---
Here are the plotting functions that used in the project. Put them here to increase the readability of the notebookv

In [ ]:
def plot_hist(x, col_name, title, x_label=None, histnorm="percent"):
    if x_label == None:
        x_label = col_name
    data = []
    for i in range(4):
        trace = go.Histogram(
            x=x[x.Target == i + 1][col_name],
            opacity=0.75,
            histnorm=histnorm,
            name=poverty_map[i+1]
        )
        
        data.append(trace)
        
    layout = go.Layout(
        barmode='stack', 
        title=title,
        xaxis=dict(
            title=x_label
        ),
        yaxis=dict(
            title="Percentage(%)"
        )
    )
    fig = go.Figure(data=data, layout=layout)

    iplot(fig, filename='overlaid histogram')

In [ ]:
def pca_3d_plot(pca_train_df):
    data = []
    colors = ['rgb(228,26,28)','rgb(55,126,184)','rgb(77,175,74)', 'rgb(230, 230, 20)']

    for i in range(4):
        color = colors[i]
        x = pca_train_df[pca_train_df.Target == i + 1][0]
        y = pca_train_df[pca_train_df.Target == i + 1][1]
        z = pca_train_df[pca_train_df.Target == i + 1][2]

        trace = dict(
            name = poverty_map[i + 1],
            x = x, y = y, z = z,
            type = "scatter3d",    
            mode = 'markers',
            marker = dict( size=3, color=color, line=dict(width=0) ) )
        data.append( trace )

    layout = dict(
        width=800,
        height=550,
        autosize=False,
        title='Costa Rican Poverty Prediction PCA',
        scene=dict(
            xaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            yaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            zaxis=dict(
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            aspectratio = dict( x=1, y=1, z=0.7 ),
            aspectmode = 'manual'        
        ),
    )

    fig = dict(data=data, layout=layout)

    # IPython notebook
    iplot(fig, filename='costa_rican', validate=False)

In [ ]:
from sklearn.metrics import silhouette_score, make_scorer
def search_optimised_k(n_clusters_list, random_state, X):
    silhouette_score_list = []
    result_dict  = {}
    for n_clusters in n_clusters_list:
        km_predict = KMeans(random_state=random_state, n_jobs=-1, n_clusters=n_clusters).fit_predict(X)
        silhouette_score_list.append(silhouette_score(X=X, labels=km_predict, random_state=16446054))
    result_dict["silhouette_score"] = silhouette_score_list
    result_dict["n_clusters"] = n_clusters_list
    
    return result_dict
        

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score


class knn_agent:
    def __init__(self):
        self.uniform_neighbours_num = None
        self.distance_neighbours_num = None
        # === models
        self.uniform_knn = None
        self.distance_knn = None
        
        self.f1_df = None
        

    def optimise_params(self, X_train, y_train, X_dev, y_dev, max_k_num):
        uniform_f1_test_list = []
        distance_f1_test_list = []
        
        for i in range(1, max_k_num):
            uniform_knn = KNeighborsClassifier(n_neighbors=i, weights="uniform")
            distance_knn = KNeighborsClassifier(n_neighbors=i, weights="distance")

            uniform_prediction = uniform_knn.fit(X_train, y_train).predict(X_dev)
            distance_prediction = distance_knn.fit(X_train, y_train).predict(X_dev)

            uniform_f1_test_list.append(f1_score(y_true=y_dev, y_pred=uniform_prediction, average='macro'))
            distance_f1_test_list.append(f1_score(y_true=y_dev, y_pred=distance_prediction, average='macro'))

        self.f1_df = pd.DataFrame(data={"F1(uniform)": uniform_f1_test_list,
                                       "F1(distance)": distance_f1_test_list},
                                 index=range(1, max_k_num))

        # Observe from the plot we can tell that the global minimum is the uniform neighbours with total market values
        self.uniform_neighbours_num = self.f1_df.sort_values("F1(uniform)", ascending=False).head(1).index.values.item(0)

        self.distance_neighbours_num = self.f1_df.sort_values("F1(distance)", ascending=False).head(1).index.values.item(0)

        # Initialise the model with optimised k number
        self.uniform_knn = KNeighborsClassifier(n_neighbors=self.uniform_neighbours_num, weights="uniform")
        self.distance_knn = KNeighborsClassifier(n_neighbors=self.distance_neighbours_num, weights="distance")
        
        self.plot_validation_result()

    def plot_validation_result(self):
        
        self.f1_df.iplot(kind='scatter', yTitle="F1 Score", xTitle="K number", title="Optimised K Number Searching")
        
        
    # Noted that predict data need to clean before feeding
    def predict(self, X_train, y_train, X_predict):
        return (self.uniform_knn.fit(X_train, y_train).predict(X_predict), self.distance_knn.fit(X_train, y_train).predict(X_predict))

    def evaluate(self, X_train, y_train, X_test, y_test):
        uniform_prediction = self.uniform_knn.fit(X_train, y_train).predict(X_test)

        distance_prediction = self.distance_knn.fit(X_train, y_train).predict(X_test)

        return (f1_score(y_true=y_test, y_pred=uniform_prediction, average="macro"),
                f1_score(y_true=y_test, y_pred=distance_prediction, average="macro"))

In [ ]:
def generate_final_predict_df(input_prediction):
    input_prediction = input_prediction.astype(int)
    X_predict["Target"] = input_prediction

    tmp_household_predict_df = X_predict.set_index("idhogar")
    tmp_X_predict = all_predict_df.loc[all_predict_df["parentesco1"] == 1, :].set_index("idhogar")
    tmp_X_predict.loc[tmp_household_predict_df.index, "Target"] = tmp_household_predict_df.Target
    tmp_X_predict = tmp_X_predict.set_index("Id")
    tmp_all_predict_df = all_predict_df.set_index("Id")
    tmp_all_predict_df.loc[tmp_X_predict.index, "Target"] = tmp_X_predict.Target
    tmp_all_predict_df["Target"] = tmp_all_predict_df["Target"].fillna(1).astype(int)
    tmp_all_predict_df

    tmp_all_predict_df["Target"] = tmp_all_predict_df["Target"].fillna(1).astype(int)
    final_predict_df = tmp_all_predict_df.reset_index()[["Id", "Target"]]
    return final_predict_df


In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
def plot_train_test(x, y, data, hue, title=None, x_label=None, y_label=None):
    
    # First check if y is a list
    y_is_list = True if isinstance(y, list) else False
    
    x_data_list = []
    y_data_list = []
    
    # Check if the hue is specified
    hue_list = data.loc[:, hue].unique()
    for cat in hue_list:
        x_data = data[data[hue] == cat][x]
        x_data_list.append(x_data)
        y_data = data[data[hue] == cat][y]
        y_data_list.append(y_data)

    data = []
    style_list = [None, 'dash', 'dot']
    colour_list = ['rgb(22, 96, 167)', 'rgb(205, 12, 24)']
    for x_data, y_data, cat, colour in zip(x_data_list, y_data_list, hue_list, colour_list):
        if(isinstance(y_data, pd.Series)):
            trace = go.Scatter(x=x_data, y=y_data, mode='lines+markers', name=cat )
            data.append(trace)
        elif(isinstance(y_data, pd.DataFrame)):
            # Means that y is a list of column names
            for col, style in zip(y, style_list):
                y_series = y_data[col]
                trace = go.Scatter(x=x_data, y=y_series, name= '+'.join([str(cat), col]),
                                   line=dict(
                                       dash = style,
                                       color=colour,
                                   )
                                  )
                data.append(trace)
    
    if x_label == None:
        x_label = x
    if y_label == None:
        if y_is_list:
            y_label = '+'.join(y)
        else:
            y_label = y
            
    layout = dict(
        title=title,
        xaxis=dict(title=x_label),
        yaxis=dict(title=y_label)
    )
    
    fig = dict(data=data, layout=layout)
    iplot(fig)

In [ ]:
from math import ceil
def subplot_test_train(x, y, z, data,hue=None, title=None, x_label=None, y_label=None):
    
    y_is_list = True if isinstance(y, list) else False
    
    if x_label == None:
        x_label = x
    if y_label == None:
        if y_is_list:
            y_label = 'f1_macro score'
        else:
            y_label = y
    
    subplot_val_list = data.loc[:, z].unique()
    subplot_num = len(subplot_val_list)
    
    subplot_titles=[]
    for val in subplot_val_list:
        subplot_titles.append(': '.join([z, str(val)]))
    
    fig = tools.make_subplots(cols=2, rows=ceil(subplot_num/2), subplot_titles=subplot_titles, print_grid=False)
    
    original_data = data
    for i, val in enumerate(subplot_val_list):
        
        data = original_data[original_data[z] == val]
        xaxis_index = 'xaxis' + str(i + 1)
        yaxis_index = 'yaxis' + str(i + 1)
        
        x_data_list = []
        y_data_list = []
        
        if hue != None:
            hue_list = data.loc[:, hue].unique()
            for cat in hue_list:
                x_data = data[data[hue] == cat][x]
                x_data_list.append(x_data)
                y_data = data[data[hue] == cat][y]
                y_data_list.append(y_data)     
        else:
            hue_list = []
    
        style_list = [None, 'dash', 'dot']
        colour_list = ['rgb(22, 96, 167)', 'rgb(205, 12, 24)', 'rgb(12, 205, 24)' ]

        for x_data, y_data, cat, colour in zip(x_data_list, y_data_list, hue_list, colour_list):
            if(isinstance(y_data, pd.Series)):
                trace = go.Scatter(x=x_data, y=y_data, mode='lines+markers' )
                fig.append_trace(trace, i//2 + 1, (i%2 + 1))
            elif(isinstance(y_data, pd.DataFrame)):
                # Means that y is a list of column names
                for col, style in zip(y, style_list):
                    y_series = y_data[col]
                    trace = go.Scatter(x=x_data, y=y_series, name= '+'.join([cat, col]),
                                       line=dict(
                                           dash=style,
                                           color=colour,
                                       )
                                      )
                    fig.append_trace(trace, i//2 + 1, (i%2 + 1))
                    if i == subplot_num - 1 or i == subplot_num - 2:
                        fig['layout'][xaxis_index].update(title = x_label)
                    if i%2 == 0:
                        fig['layout'][yaxis_index].update(title = y_label)

    fig['layout'].update(title='SVC Optimised Parameters Searching')
    iplot(fig)
#     for val in subplot_val_list:
        

In [ ]:
def ada_report(ada_models, X_train, y_train, X_test, y_test):
    
    ada_tree_real, ada_tree_discrete = ada_models
    
    real_test_f1 = []
    real_train_f1 = []
    discrete_test_f1 = []
    discrete_train_f1 = []
    
    trees_discrete_num = len(ada_tree_discrete)
    trees_real_num = len(ada_tree_real)
    
    for real_test_predict, real_train_predict, discrete_test_predict, discrete_train_predict in zip(
        ada_tree_real.staged_predict(X_test),
        ada_tree_real.staged_predict(X_train),
        ada_tree_discrete.staged_predict(X_test),
        ada_tree_discrete.staged_predict(X_train)
    ):
        real_test_f1.append(
            f1_score(y_pred=real_test_predict, y_true=y_test, average='macro')
        )
        
        real_train_f1.append(
            f1_score(y_pred=real_train_predict, y_true=y_train, average='macro')
        )
        
        discrete_test_f1.append(
            f1_score(y_pred=discrete_test_predict, y_true=y_test, average='macro')
        )
        
        discrete_train_f1.append(
             f1_score(y_pred=discrete_train_predict, y_true=y_train, average='macro')
        )
        
    f1s = [
        real_test_f1,
        real_train_f1,
        discrete_test_f1,
        discrete_train_f1, 
    ]
    labels = ["SAMME.R: Test", "SAMME.R: Train", "SAMME: Test", "SAMME: Train"]
    colours = ['rgb(22, 96, 167)', 'rgb(22, 96, 167)', 'rgb(205, 12, 24)', 'rgb(205, 12, 24)']
    styles = [None, "dash", None, "dash"]
        
    fig = tools.make_subplots(cols=1, rows=3, print_grid=False, shared_xaxes=True)
    
    for f1, label, colour, style in zip(f1s, labels, colours, styles):
        trace = go.Scatter(
            x=np.arange(1, trees_real_num, 1),
            y=f1,
            name= label,
            line=dict(
                color=colour,
                dash=style
            )
        )
        
        fig.append_trace(trace, 1, 1)
    
    fig["layout"]["yaxis1"].update(title="F1 Macro Score")
    
    trace_real_error = go.Scatter(
        x=np.arange(1, trees_real_num, 1),
        y=ada_tree_real.estimator_errors_,
        name="SAMME.R: Error",
#         line=dict(
#             color='rgb(22, 96, 167)',
#             width=3
#         )
    )
    
    trace_discrete_error = go.Scatter(
        x=np.arange(1, trees_real_num, 1),
        y=ada_tree_discrete.estimator_errors_,
        name="SAMME: Error",
#         line=dict(
#             color='rgb(205, 12, 24)',
#             width=3
#         )
    )
    
    fig.append_trace(trace_real_error, 2, 1)
    fig.append_trace(trace_discrete_error, 2, 1)
    fig["layout"]["yaxis2"].update(title="Error")
    
    trace_real_weights = go.Scatter(
        x=np.arange(1, trees_real_num, 1),
        y=ada_tree_real.estimator_weights_,
        name="SAMME.R: Weights",
    )
    
    trace_discrete_weights = go.Scatter(
        x=np.arange(1, trees_real_num, 1),
        y=ada_tree_discrete.estimator_weights_,
        name="SAMME: Weights",
    )
    
    fig.append_trace(trace_real_weights, 3, 1)
    fig.append_trace(trace_discrete_weights, 3, 1)
    fig["layout"]["yaxis3"].update(title="Weights")
    
    
    fig["layout"].update(title="Adaboost Discrete and Real Algorithm Comparison")
    iplot(fig)

In [ ]:
def plot_features(feature_importances, index, most_important=True):
    indices = np.argsort(feature_importances)[::-1]
    indices = indices[:index]

    # Visualise these with a barplot
    plt.subplots(figsize=(20, 15))
    g = sns.barplot(y=X.iloc[:, 3:].columns[indices], x = lgb_clf.feature_importances_[indices], orient='h')
    g.set_xlabel("Relative importance",fontsize=20)
    g.set_ylabel("Features",fontsize=20)
    g.tick_params(labelsize=15)
    g.set_title("LightGBM feature importance", fontsize=20);

In [ ]:
def plot_submission(df):
    trace = go.Scatter(x=df.index, y=df["public score"], mode='lines+markers', text=df["Note"])
    
    annotations_list = []
    for index, row in df.iterrows():
        annotations_list.append(
            dict(
                x=index,
                y=row[2],
                xref='x',
                yref='y',
                text=row[0],
                showarrow=True,
                arrowhead=7
            )
        )
        
    layout = go.Layout(annotations=annotations_list, title="Hover Over The Points To See My Note",
                       xaxis=dict(title="Submission Sequence"),
                       yaxis=dict(title="Finial F1 Score")
                      )
    
    fig = go.Figure(data=[trace], layout=layout)

    iplot(fig)

In [ ]:
poverty_map = {
    1:"extreme poverty", 
2 : "moderate poverty" ,
3 : "vulnerable households", 
4 : "non vulnerable households"
}

In [ ]:
raw_train_df = pd.read_csv(prefix + "train.csv")

In [ ]:
raw_predict_df = pd.read_csv(prefix + "test.csv")

In [ ]:
example_predict_df = pd.read_csv(prefix + "sample_submission.csv")

Concatenate two data sets to avoid redundant preprocessing. Split it before the machine learning procedure starts.

In [ ]:
raw_predict_df = pd.concat([raw_predict_df, pd.Series(np.nan, name='Target')], axis=1)

In [ ]:
raw_all_df = pd.concat([raw_train_df, raw_predict_df], ignore_index=True)

## Data Preprocessing
---
For the real-life data set, it is complex and incomplete. A preprocessing procedure will be crucial before any machine learning process takes place.

### Data Integrity Checking

In [ ]:
raw_all_df["edjefe"] = raw_all_df["edjefe"].apply(lambda x: 1 if x == "yes" else x).apply(lambda x: 0 if x == "no" else x)

raw_all_df["edjefa"] = raw_all_df["edjefa"].apply(lambda x: 1 if x == "yes" else x).apply(lambda x: 0 if x == "no" else x)

raw_all_df["dependency"] = raw_all_df["dependency"].apply(lambda x: 1 if x == "yes" else x).apply(lambda x: 0 if x == "no" else x)

raw_all_df["edjefe"] = pd.to_numeric(raw_all_df["edjefe"])

raw_all_df["edjefa"] = pd.to_numeric(raw_all_df["edjefa"])

raw_all_df["dependency"] = pd.to_numeric(raw_all_df["dependency"])

> ### Fill the NaN Value

In [ ]:
raw_all_df.isnull().sum()[raw_all_df.isnull().sum() > 0]

As we can see, `v2a1` which is monthly rent payment and `rez_esc` which is the years behind in school have a significant number of null values. To drop those columns will be an option.

`v18q1` is the number of ipads a household owns, after checking, we found that `NaN` in that value actually represent 0. Then we can drop the `v18q` due to `v18q1` is encoded the information of `v18q`

In [ ]:
clean_all_df = raw_all_df.drop(["v2a1", "rez_esc"], axis=1)
clean_all_df["v18q1"].fillna(0, inplace=True)

In [ ]:
clean_all_df.isnull().sum()[clean_all_df.isnull().sum() > 0]

### Remove the Redundant Features
---
In the data set, there are features which are the calculated from the other features by some math formula.
Thanks to [Will](https://www.kaggle.com/willkoehrsen/a-complete-introduction-and-walkthrough)'s exellent kernel that finds out the category of features.

In [ ]:
id_col = ['Id', 'idhogar', 'Target']

In [ ]:
ind_bool_col = ['v18q', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 
            'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 
            'parentesco1', 'parentesco2',  'parentesco3', 'parentesco4', 'parentesco5', 
            'parentesco6', 'parentesco7', 'parentesco8',  'parentesco9', 'parentesco10', 
            'parentesco11', 'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 
            'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 
            'instlevel9', 'mobilephone',]

ind_ordered_col = ['escolari', 'age']

In [ ]:
hh_bool_col = ['hacdor', 'hacapo', 'v14a', 'refrig', 'paredblolad', 'paredzocalo', 
           'paredpreb','pisocemento', 'pareddes', 'paredmad',
           'paredzinc', 'paredfibras', 'paredother', 'pisomoscer', 'pisoother', 
           'pisonatur', 'pisonotiene', 'pisomadera',
           'techozinc', 'techoentrepiso', 'techocane', 'techootro', 'cielorazo', 
           'abastaguadentro', 'abastaguafuera', 'abastaguano',
            'public', 'planpri', 'noelec', 'coopele', 'sanitario1', 
           'sanitario2', 'sanitario3', 'sanitario5',   'sanitario6',
           'energcocinar1', 'energcocinar2', 'energcocinar3', 'energcocinar4', 
           'elimbasu1', 'elimbasu2', 'elimbasu3', 'elimbasu4', 
           'elimbasu5', 'elimbasu6', 'epared1', 'epared2', 'epared3',
           'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3', 
           'tipovivi1', 'tipovivi2', 'tipovivi3', 'tipovivi4', 'tipovivi5', 
           'computer', 'television', 'lugar1', 'lugar2', 'lugar3',
           'lugar4', 'lugar5', 'lugar6', 'area2' , 'area1'] # redundant

hh_ordered_col = [ 'rooms', 'r4h1', 'r4h2', 'r4h3', 'r4m1','r4m2','r4m3', 'r4t1',  'r4t2', 
               'v18q1', 'hogar_nin', 'tamhog','tamviv','hhsize', 'r4t3', 'hogar_total',# redundant
              'hogar_adul','hogar_mayor',  'bedrooms', 'qmobilephone']

hh_cont_col = ['dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding']

In [ ]:
sqr_col = ['SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 
        'SQBhogar_nin', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq']

Sqaure of the features we have can be easily calculated afterwards. There is no need to keep them as a separated column.

In [ ]:
clean_all_df.drop(sqr_col, axis=1, inplace=True)

Next step is to reoder the columns to make all the ID columns at the begining.

In [ ]:
cols_list = id_col + ind_bool_col + ind_ordered_col + hh_bool_col + hh_ordered_col + hh_cont_col

In [ ]:
clean_all_df = clean_all_df[cols_list]

In [ ]:
clean_all_df.isnull().sum()[clean_all_df.isnull().sum() > 0]

`meaneduc` is the average years of education for adults (18+). The `NaN` value is unclear in this feature. As the portion of `NaN` value in this feature, we will calculate the median `meaneduc` value within same target and assign it to the `NaN`.

In [ ]:
temp_all_df = pd.concat([clean_all_df, pd.Series(np.argmax(clean_all_df.loc[:, "instlevel1" : "instlevel9"].values, axis=1), name="eduindex")], axis=1)
edu_median_dict = temp_all_df.groupby("eduindex")["meaneduc"].median().to_dict()

In [ ]:
def fill_na(x):
    if pd.isnull(x["meaneduc"]):
        x.loc["meaneduc"] = edu_median_dict[x.loc["eduindex"]]
    return x

In [ ]:
temp_all_df = temp_all_df.apply(fill_na, axis=1)

In [ ]:
# edu_list = clean_all_df.loc[:, "instlevel1" : "instlevel9"].columns
clean_all_df = temp_all_df.drop("eduindex", axis=1)

Due to only the heads of household will be used in scoring, we should make a seperate dataset for heads only. The rest data set can be used as a support features for heads.

In [ ]:
heads_all_df = clean_all_df[clean_all_df.parentesco1 == 1]

### Remove the High Correlated Features

In [ ]:
corr_matrix =  clean_all_df[clean_all_df.Target.notnull()].corr().abs()
np.fill_diagonal(corr_matrix.values, np.NaN)
sorted_corr_series = corr_matrix.unstack().sort_values(kind="quicksort", ascending=False)

In [ ]:
sorted_corr_series[sorted_corr_series > 0.90].head(3)

| Feature      | Describe           | 
| ------------- |-------------| 
| tamhog      | size of the household | 
| hhsize      | household size      | 
| hogar_total | # of total individuals in the household      |
|r4t3| Total persons in the household |
|tamviv|number of persons living in the household|

We will keep `tamviv` and drop all other redundant columns.

In [ ]:
rm_cols = ["tamhog", "r4t3", "hhsize", "hogar_total", "male", "area1"]

In [ ]:
clean_all_df.drop(rm_cols, axis=1, inplace=True)

In [ ]:
for col in rm_cols:
    for _list in [ind_bool_col, ind_ordered_col, hh_bool_col, hh_cont_col, hh_ordered_col]:
        if col in _list:
            _list.remove(col)

In the `cielorazo` column, the values is not consistent. There are mistake in it. We redefine it as the mark of people who don't have a roof.

In [ ]:
roof_list = ["techozinc", "techoentrepiso", "techocane", "techootro", "cielorazo"]

clean_all_df["cielorazo"] = clean_all_df[roof_list[:-1]].sum(axis=1).map({1:0, 0:1})

## Household Aggregation
---
Heads of household approach reach its limit. Try the approach of group member of a household and train the models.

This tell us that there might be the member in a same household might get different targets, this is not fit for the specification of the competition. Since the targets of  the members within a same household should be labeled as same as the head of household.

In [ ]:
no_head_household_ids = clean_all_df.groupby("idhogar").sum()[clean_all_df.groupby("idhogar").sum()["parentesco1"] == 0].index

We found out that the member inside those household dosen't have a head, which is considered not valid data. Drop them all

In [ ]:
clean_drop_no_head_df = clean_all_df.copy()
for x in no_head_household_ids:
    clean_drop_no_head_df.drop(clean_drop_no_head_df.loc[clean_drop_no_head_df.idhogar == x, :].index, axis=0, inplace=True)

In [ ]:
clean_heads_df = clean_drop_no_head_df.loc[clean_drop_no_head_df.parentesco1 == 1]

heads_dict = clean_heads_df[["idhogar", "Target"]].set_index("idhogar").to_dict()["Target"]

In [ ]:
def synchronise_diff_target(x):
    x.Target = heads_dict[x.idhogar]
    return x

clean_drop_no_head_df = clean_drop_no_head_df.apply(synchronise_diff_target, axis=1)

In [ ]:
aggregation_list = ["max", "sum", "min", "mean", "std"]

In [ ]:
hh_agg_features = clean_drop_no_head_df[["idhogar"] + ind_bool_col + ind_ordered_col].groupby("idhogar").agg(aggregation_list)
hh_agg_features.columns = ["_".join(x) for x in hh_agg_features.columns.ravel()]

hh_agg_features = hh_agg_features.fillna(0)

In [ ]:
clean_heads_df = clean_drop_no_head_df[clean_drop_no_head_df.parentesco1 == 1].loc[:, id_col + hh_bool_col + hh_cont_col + hh_ordered_col].set_index("idhogar")

clean_heads_df = pd.concat([clean_heads_df, hh_agg_features], axis=1, sort=False).reset_index().rename(columns={"index":'idhogar'})

If you are trained with tree algorithm, one hot encoding features need to be decoded.

In [ ]:
X = clean_heads_df[clean_heads_df.Target.notna()].reset_index(drop=True)
y = X.Target
X_predict = clean_heads_df[clean_heads_df.Target.isna()].reset_index(drop=True)

In [ ]:
all_predict_df = clean_all_df[clean_all_df.Target.isnull()]

## EDA
---
There are quite a lot of features in the dataset. Let's take a look at the features that are import(judged by boosting algorithms) and features that are not that crucial.

In [ ]:
plot_hist(X, "age_mean", title="Average Age In Same Household" ,x_label="Average age")

In [ ]:
plot_hist(X, "escolari_mean", x_label="Average years of schooling", title="Average Years of Schooling In Same Household")

Above two features are relatively good features. Especially in average years of schooling feature, we can clear see a rough pattern that _non vulnerable household_ will normally have a longer average years of schooling while the majority of poverty's education time are less than 10 years.

In [ ]:
plot_hist(X, "hacdor", x_label="Overcrowding by bedrooms", title="If the Bedrooms are Overcrowded")

This is an unimportant feature example. From the plot above we can see that the lion's share of all four targets are not overcrowding in their bedrooms. Thus, this feature will contribute little to the final prediction.

## Data Normalisation

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X.iloc[:, 3:], X_predict.iloc[:, 3:] = scaler.fit_transform(X.iloc[:, 3:]), scaler.fit_transform(X_predict.iloc[:, 3:])

## PCA
Before the machine learning process taking place, a visualisation of the training set can be intuitive. We use the **PCA** technique to visualise the data set.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
numeric_train_df = X.iloc[:, 3:]
label_series =  X.Target.reset_index(drop=True)

In [ ]:
pca = PCA(n_components=3, whiten=True, svd_solver='auto', random_state=166446054) 

In [ ]:
pca_train_array = pca.fit_transform(numeric_train_df)

In [ ]:
pca_train_df = pd.DataFrame(pca_train_array)

pca_train_df = pd.concat([pca_train_df, label_series], axis=1)

In [ ]:
pca_3d_plot(pca_train_df=pca_train_df)

Even though the 3D plot is not preferred in the data analysis, due to it is hard for a human to visualise the information that plots want to present, but the interactive 3D plots can make up this shortage and add more intuitions in the plot. Another reason is that the original dataset has more than 200 dimensions, 3D plots can preserve more information from the original dataset than 2D plots.

Above **PCA** data visualisation can tell use that the _non vulnerable households_ are separated from the rest targets.

## Clustering Analysis

In [ ]:
from sklearn.cluster import KMeans
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.pipeline import Pipeline

In [ ]:
result_dict = search_optimised_k([4, 8, 16, 32, 64, 128, 259], random_state=16446054, X=X.iloc[:, 3:])

In [ ]:
km_param_results = pd.DataFrame(result_dict)
ax = km_param_results.set_index("n_clusters").plot(figsize=(12,8), fontsize=15, title="The Silhouette Score Relationship With Number of Centroid")
ax.set_ylabel("Silhouette Score")

In [ ]:
km_predict = KMeans(n_jobs=-1, n_clusters=4, random_state=16446054).fit_predict(X.iloc[:, 3:])
km_X = pd.concat([pd.Series(km_predict, name="k_means_predict").map({0:"centroid1",
                                                                     1:"centroid2",
                                                                     2: "centroid3",
                                                                     3: "centroid4"
                                                                    }), X.iloc[:, 2:]], axis=1)

plot_hist(km_X, "k_means_predict", x_label="KMeans Prediction", title="K Means Clustering Prediction")

The result is not ideal. For each centroid, the distribution of each target is similar. Let's try using **PCA** to transform the inputs first

In [ ]:
result_dict = search_optimised_k([4, 8, 16, 32, 64, 128, 259], random_state=16446054, X=pca_train_df)
km_param_results = pd.DataFrame(result_dict)
ax = km_param_results.set_index("n_clusters").plot(figsize=(12,8), fontsize=15, title="The Silhouette Score Relationship With Number of Centroid")
ax.set_ylabel("Silhouette Score")

In [ ]:
km_predict = KMeans(n_jobs=-1, n_clusters=4, random_state=16446054).fit_predict(pca_train_df)
km_X = pd.concat([pd.Series(km_predict, name="k_means_predict").map({0:"centroid1",
                                                                     1:"centroid2",
                                                                     2: "centroid3",
                                                                     3: "centroid4",
                                                                     4: "centroid5",
                                                                     5: "centroid6",
                                                                     6: "centroid7",
                                                                     7: "centroid8"
                                                                    }), X.iloc[:, 2:]], axis=1)

plot_hist(km_X, "k_means_predict", x_label="KMeans Prediction", title="K Means Clustering Prediction")

The result of the above clustering is impressive, the centroid 3 has clustered more than 90% extreme poverty and 80% moderate poverty. Comparing to the raw input without **PCA**, the improvement is huge. A future exploration can be designed to vary the number of components as well as the number of centroids to find an optimised result of clustering. However, the further explore is beyond the domain of this project.

# Machine Learning
---
In this section, a variety of machine learning algorithm will be explored. This project will examine the _K nearest neighbour, support vector machine, ada tree boosting and light gradient boosting algorithm_ in order. The optimised hyperparameter searching process of `KNN` algorithm is built from scratch. The reset use the [cross validation grid search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class provided by `sklearn` model to find the optimised parameters. The performance of each algorithm is evaluated.

In [ ]:
from sklearn.model_selection import train_test_split

## KNN

In [ ]:
my_knn_agent = knn_agent()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 3:], y, test_size=0.2, random_state=16446054, stratify=y)

In [ ]:
my_knn_agent.optimise_params(X_train=X_train, y_train=y_train, X_dev=X_test, y_dev=y_test, max_k_num=50)

Put the cursor on the plot we will easily locate the optimised test f1 score. The k number is 5 and with the uniform algorithm, it can achieve the 0.4 f1 score.

In [ ]:
uniform_f1, distance_f1 = my_knn_agent.evaluate(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
print("Unifrom test set f1 score: ", uniform_f1, " Distance test set f1 score: ", distance_f1)

In [ ]:
uniform_prediction, distance_prediction = my_knn_agent.predict(X_train=X_train, y_train=y_train, X_predict=X_predict.iloc[:, 3:])

In [ ]:
# final_predict_df = generate_final_predict_df(uniform_prediction)

# final_predict_df.to_csv("knn_submission.csv", index=False)

It has been prove that the submission has nothing to do with the rows which is not the head of household.

---
Follow will use a `sklearn` approach to search for optimising hyperparameters. We will use the `GridSearchCV` approach to find the optimised hyperparameters. This is a extremely time consuming procedure. As we have found the optimised **K** number in the previous section, here we will stick with the number as benchmark to search other hyperparameters. However, if one wants to achieve the ideal optimised parameters, the search parameters should be combined, because, with different dimension of **X**, the opimised **K** might be different.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
cachedir = mkdtemp()

knn_pipe = Pipeline([('reduce_dim', PCA(random_state=16446054)), ('classify', my_knn_agent.uniform_knn)], memory=cachedir)

In [ ]:
num_features_list = np.arange(10, 200, 20)

In [ ]:
knn_param_grid = [
    {
        'reduce_dim': [PCA(random_state=16446054)],
        'reduce_dim__n_components': num_features_list,
    }, 
    {
        'reduce_dim': [SelectKBest(chi2)],
        'reduce_dim__k': num_features_list,
    }
]

reducer_labels = ['PCA', 'KBest(chi2)']

In [ ]:
knn_grid = GridSearchCV(estimator=knn_pipe, cv=5, n_jobs=4, param_grid=knn_param_grid, scoring='f1_macro')

In [ ]:
_ = knn_grid.fit(X.iloc[:, 3:], y)

In [ ]:
report(knn_grid.cv_results_)

In [ ]:
knn_params_results = pd.DataFrame(knn_grid.cv_results_)

In [ ]:
knn_params_results = pd.concat([knn_params_results.param_reduce_dim__n_components.fillna(knn_params_results.param_reduce_dim__k).rename("num_features"),
                                knn_params_results.drop(["param_reduce_dim__k", "param_reduce_dim__n_components"], axis=1)], axis=1)

In [ ]:
knn_params_results.loc[:, "param_reduce_dim"] = knn_params_results.param_reduce_dim.apply(lambda x: "PCA" if str(x).startswith("PCA") else "SelectKBest").rename("algorithm")

In [ ]:
plot_train_test("num_features", ["mean_test_score", "mean_train_score"], 
                hue="param_reduce_dim", data=knn_params_results, y_label="F1 Score",
                title="Average Train/Test F1 Score Relationship with the Number of Features")

PCA will remain the information of original data while select k best techniques will discard the unimportance. The result is not a surprise that with the low dimensionality that PCA still have a relatively high accuracy. 

In [ ]:
# Clear the cache directory when you don't need it anymore
rmtree(cachedir)

## SVM
---

In [ ]:
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
cachedir = mkdtemp()

svc_pipe = Pipeline([('reduce_dim', PCA(random_state=166446054)), ('classify', SVC(random_state=16446054, class_weight="balanced"))], memory=cachedir)

In [ ]:
svc_param_grid = [
    {
        'reduce_dim__n_components': [10, 70, 140, 200],
        'classify__kernel': ['rbf', 'poly'],
        'classify__C': [0.1, 1, 10, 50],
    }
]

In [ ]:
svc_grid = GridSearchCV(svc_pipe, cv=5, n_jobs=4, param_grid=svc_param_grid, scoring='f1_macro', refit=True)

_ = svc_grid.fit(X.iloc[:, 3:], y)

In [ ]:
report(svc_grid.cv_results_)

In [ ]:
svc_params_results = pd.DataFrame(svc_grid.cv_results_)

In [ ]:
subplot_test_train(x="param_reduce_dim__n_components", y=["mean_test_score", "mean_train_score"], z="param_classify__C", data=svc_params_results, hue="param_classify__kernel")

From the subplots, we can extract follow intuitions:
* With high dimensionality, polynomial kernel `SVC` is underfitting
* `rbf` kernel will be more stable for a varied range of dimensions of the dataset
* The highest test set score is gained with polynomial kernel and big number C(50) as well as a high dimensional dataset.

In [ ]:
rmtree(cachedir)

In [ ]:
# clf.fit(X.iloc[:, 3:], X.Target)

In [ ]:
# svc_test_predict = clf.predict(heads_test_df.iloc[:, 3:])
# print("SVC test prediction f1 score: ",f1_score(y_pred=svc_test_predict, y_true=heads_test_df.Target, average="macro"))

In [ ]:
# svc_predict = clf.predict(X_predict.iloc[:, 3:])

# final_predict = generate_final_predict_df(svc_predict)

# final_predict.to_csv("svc_submission.csv", index=False)

## AdaBoost
---
Boosting algorithm is efficient on dataset with a lot of features. As a result it achieved a great success on the facial recognition process with haar-like features which are huge in this task. Intuit


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
ada_tree_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1.5
)

ada_tree_discrete = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1.5,
    algorithm="SAMME"
)

In [ ]:
ada_tree_real.fit(X_train, y_train)
ada_tree_discrete.fit(X_train, y_train)

In [ ]:
ada_report(ada_models=(ada_tree_real, ada_tree_discrete), X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

The real boosting algorithm can achieve a good accuracy on the test set within a small iteration. However, the real boosting algorithm seems prone to overfitting due to the big gap between the train set and test set scores. But after 400 estimators , the difference between the two algorithms become small.

Afterwards, let's find out, for the boosting algorithm which can select good features out of bad features, if the more features the better.

In [ ]:
cachedir = mkdtemp()

ada_pipe = Pipeline([('reduce_dim', PCA(random_state=166446054)),
                     ('classify', AdaBoostClassifier(random_state=16446054))], memory=cachedir)

Try to evaluate the effect of dimension 

In [ ]:
ada_param_grid = [
    {
        'reduce_dim__n_components': [10, 70, 140, 259],
        'classify__algorithm': ['SAMME.R', 'SAMME'],
        'classify__learning_rate': [1],
        'classify__n_estimators': [300],
        'classify__base_estimator': [DecisionTreeClassifier(max_depth=2)]
    }
]

In [ ]:
ada_grid = GridSearchCV(ada_pipe, cv=5,
                        n_jobs=4, param_grid=ada_param_grid,
                        scoring='f1_macro', refit=True, )#verbose=20) # This can show the progress of searching

_ = ada_grid.fit(X.iloc[:, 3:], y)

In [ ]:
report(ada_grid.cv_results_)

In [ ]:
ada_param_results = pd.DataFrame(ada_grid.cv_results_)

In [ ]:
plot_train_test(x="param_reduce_dim__n_components", y=["mean_test_score", "mean_train_score"], data=ada_param_results, hue="param_classify__algorithm")

The result above demonstrates the fact that with more dimensionalities of the data set, the adaboost algorithm actually becomes prone to overfitting problem. The gap between the training set and testing set becomes larger as the dimension growing bigger. 

It is to my surprise that adaboost is still prone to high dimensional data, an appropriate  dataset preprocessing can still help to increase the performance 

In [ ]:
# real_test_predict = np.array(ada_real_test_result).mean(axis=0).round().astype(int)
# discrete_test_predict = np.array(ada_discrete_test_result).mean(axis=0).round().astype(int)
# print("Real test prediction f1 score: ",f1_score(y_pred=real_test_predict, y_true=heads_dev_df.Target, average="macro"))
# print("Discrete test prediction f1 score: ",f1_score(y_pred=discrete_test_predict, y_true=heads_dev_df.Target, average="macro"))

# real_test_predict = ada_tree_real.predict(heads_test_df.iloc[:, 3:])
# discrete_test_predict = ada_tree_discrete.predict(heads_test_df.iloc[:, 3:])
# print("Real test prediction f1 score: ",f1_score(y_pred=real_test_predict, y_true=heads_test_df.Target, average="macro"))
# print("Discrete test prediction f1 score: ",f1_score(y_pred=discrete_test_predict, y_true=heads_test_df.Target, average="macro"))

# real_predict = ada_tree_real.predict(X_predict.iloc[:, 3:]).astype(int)
# discrete_predict = ada_tree_discrete.predict(X_predict.iloc[:, 3:]).astype(int)

# real_final_predict = generate_final_predict_df(real_predict)
# discrete_finial_predict = generate_final_predict_df(discrete_predict)

# real_final_predict.to_csv("real_ada_submission.csv", index=False)

# discrete_finial_predict.to_csv("discrete_ada_submission.csv", index=False)

## LightGBM
---

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn import preprocessing

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Noted the parameters here are not optimised ones that help me to get my best score, the optimised one will take ages to train so I change it to a fast training mode with a acceptable accuracy

In [ ]:
lgb_clf = lgb.LGBMClassifier(learning_rate=0.01, n_estimators=15000, 
                             objective='multiclass', matric='logloss', boosting_type='dart',
                             num_leaves=700, max_depth=10,
                             class_weight='balanced',  silent=True, n_jobs=-1,
                             colsample_bytree =  0.93, min_child_samples = 95,  subsample = 0.96)

In [ ]:
kfold = 10
kf = StratifiedKFold(n_splits=kfold, shuffle=True)

predicts_result = []
test_result = []
for train_index, test_index in kf.split(X.iloc[:, 3:], X.Target):
    print("#"*10)
    X_train, X_val = X.iloc[:, 3:].iloc[train_index], X.iloc[:, 3:].iloc[test_index]
    y_train, y_val = X.Target.iloc[train_index], X.Target.iloc[test_index]
    lgb_clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=1000, verbose=10000) #eval_metric=f1_macro_evaluation)
    test_result.append(f1_score(y_pred=lgb_clf.predict(X_val), y_true=y_val, average="macro"))
    predicts_result.append(lgb_clf.predict(X_predict.iloc[:, 3:]))

In [ ]:
for result in test_result:
    print(result)
print("Mean test f1: ", np.mean(test_result))

In [ ]:
lgb_train_predict = lgb_clf.predict(X.iloc[:, 3:])
lgb_test_f1 = f1_score(y_pred=lgb_train_predict, y_true=X.Target, average="macro")
print("LGBM train f1: ", lgb_test_f1)

In [ ]:
# lgb_predict = np.array(predicts_result).mean(axis=0).round().astype(int)
# lgb_predict = lgb_clf.predict(X_predict.iloc[:, 3:])

# final_predict = generate_final_predict_df(lgb_predict)

# final_predict.to_csv('submission.csv', index = False)

In [ ]:
plot_features(lgb_clf.feature_importances_, index=20)

An advantage for boosting algorithm is that it can find the importance of each feature. The actually developing workflow can be circular. We can reduce the unimportant features from the dataset and run boosting algorithm again to see the difference. Another thought could be use **PCA** to merge all the unimportant features into one feature, to sum up the information from those features.

# Conclusion
---
As the real-life machine learning problem, the data wrangling part is crucial for models training phase. In this project, `NaN` value is filled thoughtfully and the highly correlated features are dropped. To merge the members within the same household can help increase the accuracy of models.

A clustering analysis combined with **PCA** gives an interesting result that one of the centroids can separate 90% extreme poverty and 80% moderate poverty. Another finding is that **PCA** can help improve the clustering result.

In the machine learning section, a variety of algorithms are evaluated to find the optimised parameters. The main trend is that as the dimensionality of data becomes larger the models become prone to overfitting problem. This rule even applies to ada boost algorithm. **PCA** is a good choice to reduce the dimension than `selectKBest` algorithm overall, especially when the reduction ratio is large. It should be included in the standard process of machine learning workflow especially for the algorithm like K nearest neighbours and support vector machine.

The boosting algorithm has the best performance among all the algorithms that evaluated in this project for this specific dataset. The **LightGBM** algorithm is especially good for this task.

# Append
---
This section record the plot of my submission and the improvement I have done in that submission

In [ ]:
submission_history = pd.read_excel("../input/competitionjournal/costa_rican_competition_result.xlsx")

submission_history.set_index("sequence", inplace=True)

plot_submission(submission_history)